# Part 3: Example application


First, we begin by constructing 


## Scope



If you have comments or questions about the notebooks, the best place to get help is through [GitHub Issues](https://github.com/TerrainBento/umami/issues).

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from io import StringIO
from itertools import product

import numpy as np

import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from plotnine import *

import holoviews as hv
hv.notebook_extension('matplotlib')

from landlab import imshow_grid
from terrainbento import Basic
from umami import Metric, Residual

In [ ]:
spec_string = """
# Create the Clock.
clock:
    start: 0
    step: 500
    stop: {duration}

# Create the Grid
grid: 
    RasterModelGrid: 
        - [50, 80]
        - xy_spacing: 100
        - fields: 
            node: 
                topographic__elevation:
                    random:
                        where: CORE_NODE
                    constant:
                        value: 100
                        
# Set up Boundary Handlers
boundary_handlers: 
    SingleNodeBaselevelHandler: 
        outlet_id: 40
        lowering_rate: -{lowering_rate}

# Parameters that control output.
output_interval: 1e3
save_first_timestep: True
output_prefix: 
    simple_application.{name}.
fields: 
    - topographic__elevation

# Parameters that control process and rates.
water_erodibility: {water_erodibility}
m_sp: 0.5
n_sp: 1.0
regolith_transport_parameter: 0.1
"""

In [ ]:
truth_duration = 1e4
truth_water_erodibility = 0.0005

lowering_rate = 100 / truth_duration

truth_params = StringIO(
    spec_string.format(duration=truth_duration,
                       water_erodibility=truth_water_erodibility,
                       lowering_rate=lowering_rate,
                       name="truth"))
np.random.seed(42)
truth = Basic.from_file(truth_params)
truth.run()

In [ ]:
ds = truth.to_xarray_dataset(time_unit='years', space_unit='meters')
hvds_topo = hv.Dataset(ds.topographic__elevation)
topo = hvds_topo.to(hv.Image, ['x', 'y'],
                    label='Truth').options(interpolation='bilinear',
                                           cmap='viridis',
                                           colorbar=True)
topo

In [ ]:
ds.close()
truth.remove_output_netcdfs()

In [ ]:
residual_string = """
z_me:
    _func: aggregate
    method: mean
    field: topographic__elevation
z_p10:
    _func: aggregate
    method: percentile
    field: topographic__elevation
    q: 10
z_wsmean:
    _func: watershed_aggregation
    field: topographic__elevation
    method: mean
    outlet_id: 40
ksw_z:
    _func: kstest_watershed
    outlet_id: 40
    field: topographic__elevation
ksw_da:
    _func: kstest_watershed
    outlet_id: 40
    field: drainage_area
ksw_s:
    _func: kstest_watershed
    outlet_id: 40
    field: topographic__steepest_slope
"""

In [ ]:
resolution = 10
durations = np.logspace(3, 5, num=resolution)
water_erodibilitys = np.logspace(-4, -2, num=resolution)

In [ ]:
out = {}
for i, (duration,
        water_erodibility) in enumerate(product(durations,
                                                water_erodibilitys)):
    lowering_rate = 100 / duration
    test_params = StringIO(
        spec_string.format(duration=duration,
                           water_erodibility=water_erodibility,
                           lowering_rate=lowering_rate,
                           name=i))
    #np.random.seed(42)
    test = Basic.from_file(test_params)
    test.run()

    test.remove_output_netcdfs()

    residual = Residual(test.grid, truth.grid)
    residual.add_from_file(StringIO(residual_string))
    residual.calculate()

    values = {name: residual.value(name) for name in residual.names}
    out[(duration, water_erodibility)] = values

In [ ]:
df = pd.DataFrame.from_dict(out, orient="index")
df.index.names = ["duration", "water_erodibility"]
df.head()

In [ ]:
df_melt = df.reset_index().melt(id_vars=["duration", "water_erodibility"])
df_melt["squared_residual"] = df_melt.value**2
df_melt.head()

In [ ]:
p1 = (ggplot(df_melt[df_melt.variable.str.startswith("ksw")],
             aes(x="duration", y="water_erodibility",
                 fill="squared_residual")) + geom_tile() +
      geom_point(aes(x=truth_duration, y=truth_water_erodibility)) +
      scale_fill_continuous(limits=[0.001, 1], trans="log10") +
      facet_wrap("~variable") + theme_bw() + scale_x_log10() +
      scale_y_log10() + coord_equal())

p2 = (
    ggplot(df_melt[df_melt.variable.str.startswith("z")],
           aes(x="duration", y="water_erodibility", fill="squared_residual")) +
    geom_tile() + scale_fill_continuous(limits=[0.001, 1000], trans="log10") +
    geom_point(aes(x=truth_duration, y=truth_water_erodibility)) +
    facet_wrap("~variable") + theme_bw() + scale_x_log10() + scale_y_log10() +
    coord_equal())

print(p1)
print(p2)

# Next steps

The next step is the final notebook in the four part introductory series: [Part 4: Application using the Discretized Misfit calculation](DiscretizedResidual.ipynb).